# Redis. Как реализовать кэширование информации

In [ ]:
%%capture
!pip install redis-server redis

In [ ]:
import redis_server

In [ ]:
!$redis_server.REDIS_SERVER_PATH --daemonize yes

In [ ]:
import redis

In [ ]:
client = redis.Redis(host = 'localhost', port=6379)

In [ ]:
client.ping()

True

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

In [ ]:
d = {'id':[1,2,3,4,5,6,7,8,9,10], 'val':['val1','val2','val3','val4','val5','val6','val7','val8','val9','val10']}

In [ ]:
dataset = pd.DataFrame(data=d)

In [ ]:
dataset.head()

,id,val
0,1,val1
1,2,val2
2,3,val3
3,4,val4
4,5,val5


In [ ]:
def create_table(name_table:str="test_data", 
                 col1:int ="id", 
                 col2:str="val",
                 path_database:str="/content/test.db") -> None:
    """Создать таблицу в БД"""
    try:
        conn = sqlite3.connect(f"{path_database}")
        cursor = conn.cursor()
        cursor.execute(f"drop table if exists {name_table};")
        cursor.execute(f"create table {name_table}({col1} integer, {col2} string);")
        conn.commit()
    except sqlite3.DatabaseError as err:
        print("Ошибка при работе с SQLite")
    finally:
        if conn:
            cursor.close()
            conn.close()
            print("Соединение с SQlite закрыто, таблица сформирована")

In [ ]:
create_table()

Соединение с SQlite закрыто, таблица сформирована


In [ ]:
def load_data(path_database:str="/content/test.db",
              table_name:str = 'test_data',
              col1:int ="id", 
              col2:str="val",
              dataset:pd.DataFrame = dataset) -> None:
    """Записать данные в БД"""
    try:
        conn = sqlite3.connect(path_database)
        cursor = conn.cursor()
        query = f"INSERT INTO {table_name} ({col1}, {col2}) values(?, ?);"
        data = list(dataset.to_records(index=False))
        cursor.executemany(query, data)
        conn.commit()
    except sqlite3.DatabaseError as err:
        print("Ошибка при работе с SQLite")
    finally:
        if conn:
            cursor.close()
            conn.close()
            print("Соединение с SQlite закрыто, данные добавлены")

In [ ]:
load_data()

Соединение с SQlite закрыто, данные добавлены


In [ ]:
def select_sqlite(sql, path_database:str="/content/test.db")->pd.DataFrame:
    conn = sqlite3.connect(path_database)
    return pd.read_sql(sql, conn)

In [ ]:
sql = """select * from test_data limit 5"""

In [ ]:
select_sqlite(sql)

,id,val
0,1,val1
1,2,val2
2,3,val3
3,4,val4
4,5,val5


In [ ]:
def get_value_from_db(id:int, 
            path_database:str="/content/test.db",
            table_name:str = 'test_data') -> str:
    """Получить данные из БД"""
    try:
        conn = sqlite3.connect(path_database)
        cursor = conn.cursor()
        query = f"SELECT t.val FROM {table_name} as t WHERE t.id={id};"
        cursor.execute(query)
        result = cursor.fetchone()[0]
        return result
        # conn.commit()
    except sqlite3.DatabaseError as err:
        print("Ошибка при работе с SQLite")
    finally:
        if conn:
            cursor.close()
            conn.close()
            # print("Соединение с SQlite закрыто")

In [ ]:
print(get_value_from_db(1))

val1


In [ ]:
def get_value_from_cache(id:int,redis_client=client)->str:
  """Получить значение из Redis"""
  cache_value = redis_client.get(id)
  if cache_value is not None:
    cache_value = cache_value.decode("utf-8")
  redis_client.close()
  return cache_value

In [ ]:
def write_value_to_cache(id:int,val:str,redis_client=client,expire:int=100)->None:
  """Записать значение в кэш"""
  redis_client.set(id,val,ex=expire)
  redis_client.close()

In [ ]:
def main(id:int)->str:
  """Получение актуального значения"""
  val = get_value_from_cache(id=id)
  if val is not None:
    print("Значение получено из кэша")
    return val
  else:
    val_ = get_value_from_db(id)
    write_value_to_cache(id,val_)
    print("Значение получено из БД")
    return val_

In [ ]:
main(8)

Значение получено из БД


'val8'